In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Suppress TensorFlow warnings (optional)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

# Define model parameters
img_height, img_width = 224, 224
batch_size = 32
epochs_per_cycle = 5  # Number of epochs per training cycle
num_classes = 8
target_accuracy = 0.70  # Desired accuracy

# Check if the dataset directory exists
train_dir = r'C:\Users\sarth\Desktop\project\skin-disease-datasaet\train_set'
test_dir = r'C:\Users\sarth\Desktop\project\skin-disease-datasaet\test_set'

print("Checking if train directory exists:", os.path.exists(train_dir))
print("Checking if test directory exists:", os.path.exists(test_dir))

if not os.path.exists(train_dir):
    print("The train directory does not exist or the path is incorrect.")
    exit()

if not os.path.exists(test_dir):
    print("The test directory does not exist or the path is incorrect.")
    exit()

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)  # Use 20% of the data for validation

test_datagen = ImageDataGenerator(rescale=1./255)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Function to create a CNN model
def create_model():
    model = models.Sequential([
        layers.Input(shape=(img_height, img_width, 3)),
        layers.Conv2D(16, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_model()

# Training loop
best_accuracy = 0.0

while best_accuracy < target_accuracy:
    print(f"Starting training cycle with target accuracy: 80%")
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs_per_cycle,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size)

    current_accuracy = history.history['val_accuracy'][-1]
    print(f"Validation accuracy after this cycle: {current_accuracy:.4f}")

    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        # Save the best model
        model.save('best_skin_disease_model.h5')
        print(f"New best model saved with accuracy: {best_accuracy:.4f}")

    if best_accuracy >= target_accuracy:
        print(f"Target accuracy of {target_accuracy * 100}% achieved.")
        break

# Evaluate the best model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Final Test accuracy: {test_accuracy * 100:.2f}%")

print("Training completed.")

# ------------------------- GRAD-CAM Implementation -------------------------

# Function to generate Grad-CAM heatmap
def make_gradcam_heatmap(model, img_array, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Compute gradients of the top predicted class for the convolutional output
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map by its gradient
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize the heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Function to superimpose heatmap on the original image
def superimpose_heatmap(img_path, heatmap, intensity=0.4):
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap * intensity + img
    return superimposed_img

# Load the image and preprocess it for Grad-CAM
img_path = 'test_image.jpg'  # Replace with your test image path
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Specify the last convolutional layer's name
last_conv_layer_name = 'conv2d_2'  # Adjust according to your model's architecture

# Generate the Grad-CAM heatmap
heatmap = make_gradcam_heatmap(model, img_array, last_conv_layer_name)

# Display the heatmap
plt.matshow(heatmap)
plt.show()

# Superimpose the heatmap on the original image
superimposed_img = superimpose_heatmap(img_path, heatmap)
cv2.imwrite('gradcam_output.jpg', superimposed_img)

# Show the superimposed image
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()

print("Grad-CAM visualization completed.")


ModuleNotFoundError: No module named 'cv2'